<a href="https://colab.research.google.com/github/junior8801/Web_Scraping_CVM_Py_Selenium/blob/main/Web_Scraping_CVM_Py_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Web Scraping Composição do Índice Bovespa

Fonte: https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br


##Bibliotecas

In [1]:
!pip install -q kora

In [2]:
!pip install -U plotly

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


In [3]:
from kora.selenium import wd
import pandas as pd
import numpy as np
from google.colab.data_table import DataTable
from time import sleep
import plotly.express as px

##Loading
Baixar composição do Ibovespa

In [9]:
tempo = 10
url_base = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br'
wd.get(url_base)
sleep(tempo)
wd.find_element_by_id('segment').send_keys("Setor de Atuação")
sleep(tempo)
wd.find_element_by_link_text('Download').click()
sleep(tempo*3)
arquivos = !ls -1t *.csv

In [33]:
ibov = pd.read_csv(arquivos[0],sep=';',encoding='ISO-8859-1',skipfooter=2,engine='python',thousands='.',decimal=',',header=1,index_col=False)

##Preparação dos Dados
Separar a informação de setor e subsetor

In [34]:
ibov['SubSetor'] = ibov['Setor'].apply(lambda s: s[s.rfind("/")+1:].strip())
ibov['Setor'] = ibov['Setor'].apply(lambda s: s[:s.rfind("/")].strip())

Ajuste setores duplicados

In [40]:
ibov.loc[ibov.Setor == 'Cons N  Básico' ,  'Setor'] = 'Consumo Não-Cíclico'
ibov.loc[ibov.Setor == 'Cons N Cíclico',  'Setor'] = 'Consumo Não-Cíclico'
ibov.loc[ibov.Setor == 'Financ e Outros',  'Setor'] = 'Financeiro e Outros'


In [43]:
fig = px.sunburst(data_frame=ibov, path=['Setor','SubSetor','Código'], values='Part. (%)', height=900)
fig.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:.2f}%')
fig.show()

In [44]:
fig = px.treemap(data_frame=ibov, path=['Setor','SubSetor','Código'], values='Part. (%)', height=900)
fig.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:.2f}%')
fig.show()